In [31]:
import numpy as np
import pandas as pd
import os, itertools, csv
from bs4 import BeautifulSoup
import re
from pytrends.request import TrendReq
from datetime import *
import time

# load the dataset
dataset = pd.read_csv('./data/test_2.csv')
# dataset = dataset.drop(columns=['Page content', 'article', 'title', 'article_clean', 'title_clean'])
dataset = dataset.drop(columns=['# of audio', '# of video', 'is_sun', 'is_sat', 'Unnamed: 0'])
dataset.head()

# p = dataset[dataset.loc[:,"Popularity"] > 0]
# unp = dataset[dataset.loc[:,"Popularity"] < 0]
# unp.head()

# # X = dataset.drop(columns=['Popularity'])
# y = dataset['Popularity']

,Page content,is_weekend,is_mon,article,title,article_clean,title_clean,num_char,num_words,num_sentence,unique_rate,nonstopword_rate,title_freq,# of img,# of link,# of total
0,"<html><head><div class=""article-info""><span cl...",0,1,Note to humanity: One Direction fandom ai...,Soccer Star Gets Twitter Death Threats After T...,note humanity one direction fandom nothin mess...,soccer star get twitter death threat tackling ...,2294,382,37,0.704301,0.590476,0.075916,1,30,31
1,"<html><head><div class=""article-info""><span cl...",0,0,Shortly after announcing a hardware upgrade f...,Google Glass Gets an Accessory Store,shortly announcing hardware upgrade google gla...,google glass get accessory store,489,84,6,0.696203,0.530201,0.178571,3,13,16
2,"<html><head><div class=""article-info""><span cl...",0,0,"Well, that was quick. Just hours after going ...",OUYA Gaming Console Already Sold Out on Amazon,well quick hour going sale u canada uk ouya ga...,ouya gaming console already sold amazon,620,105,13,0.808081,0.589286,0.142857,2,13,15
3,"<html><head><div class=""article-info""><span cl...",0,0,Between Two Ferns: Oscar Buzz Edition Part 1...,'Between Two Ferns' Mocks Oscar Nominees,two fern oscar buzz edition part zach galifian...,two fern mock oscar nominee,670,113,7,0.787037,0.666667,0.097345,1,15,16
4,"<html><head><div class=""article-info""><span cl...",0,0,Ever since The Hurt Locker it seems like ...,'American Sniper' Trailer: Looks Like Eastwood...,ever since hurt locker seems like military the...,american sniper trailer look like eastwood may...,835,146,10,0.839416,0.608889,0.102740,1,10,11


In [2]:
p = p.reset_index(drop=True)
unp = unp.reset_index(drop=True)
p.head()

,Popularity,Page content,is_weekend,is_mon,article,title,article_clean,title_clean,num_char,num_words,num_sentence,unique_rate,nonstopword_rate,title_freq,# of img,# of link,# of total
0,1,"<html><head><div class=""article-info""><span cl...",0,0,Google took a stand of sorts against patent-l...,Google's New Open Source Patent Pledge: We Won...,google took stand sort patent lawsuit theater ...,google new open source patent pledge sue unles...,1147,193,14,0.584699,0.592233,0.290155,2,18,20
1,1,"<html><head><div class=""article-info""><span cl...",0,0,You've spend countless hours training to be a...,Ballin': 2014 NFL Draft Picks Get to Choose Th...,spend countless hour training american footbal...,ballin nfl draft pick get choose walk music,3608,642,90,0.694006,0.466176,0.042056,2,11,13
2,1,"<html><head><div class=""article-info""><span cl...",0,1,"First thing's first, this is hilarious. A sh...",6-Second Vine Video Is the Funniest Parody of ...,first thing first hilarious short parody video...,second vine video funniest parody iggy azalea ...,425,79,6,0.901408,0.612069,0.151899,1,11,12
3,1,"<html><head><div class=""article-info""><span cl...",0,1,The days of struggling to insert a crinkled ...,Vending Machine Uses Cloud Technology to Perso...,day struggling insert crinkled dollar bill ven...,vending machine u cloud technology personalize...,1132,177,14,0.758824,0.576271,0.135593,1,14,15
4,1,"<html><head><div class=""article-info""><span cl...",0,0,"Who is Don Draper without sweat on his brow, ...",If Caricature Artists Animated 'Mad Men',draper without sweat brow drink hand cloud smo...,caricature artist animated mad men,577,94,5,0.640449,0.395556,0.085106,8,9,17


In [36]:
res = []

for i in range(0, dataset.shape[0]):
    merge_dummy = []
    merge_dummy.append(dataset['title_clean'][i])
    merge_dummy.append(dataset['article_clean'][i])
    res.append(" ".join('%s' %id for id in merge_dummy))

dataset["dummy"] = res

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    dataset['dummy'], y, test_size=0.3, random_state=0)

X_train = np.array(X_train)
X_test = np.array(X_test)

In [23]:
import numpy as np
import scipy as sp
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

pipe = Pipeline([('vect', TfidfVectorizer(ngram_range=(1,2))), 
                  ('clf', LogisticRegression(n_jobs=4))])

pipe.fit(X_train, y_train)

C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=

In [24]:
from sklearn.metrics import accuracy_score

y_pred = pipe.predict(X_test)
score = accuracy_score(y_test, y_pred)
print('Accuracy: {}'.format(score))

Accuracy: 0.5285180272518992


In [25]:
import numpy as np
import scipy as sp
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

pipe = Pipeline([('vect', TfidfVectorizer(ngram_range=(1,2))), 
                  ('clf', LinearRegression())])

pipe.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbo

In [37]:
from sklearn.metrics import accuracy_score

y_pred = pipe.predict(np.array(dataset["dummy"]))
# score = accuracy_score(y_test, y_pred)
# print('Accuracy: {}'.format(score))
y_pred
list(y_pred)

[-0.11084953288862692,
 0.28697774341588983,
 -0.2797980865720814,
 0.1412730323398346,
 -0.6205108923183266,
 -0.1234663081249425,
 -0.20297773794954815,
 0.8785206076239586,
 -0.08400529819708848,
 0.20792215992065952,
 0.14294617281682395,
 0.06147669947030061,
 0.4407090274633552,
 -0.3843674254733705,
 -0.055044703493376576,
 -1.1824687279848325,
 0.6238635317184917,
 0.27402217650072325,
 -0.7760211777225164,
 0.7082473181053252,
 -0.4439598937289746,
 0.10962336677196652,
 -0.40808528129048716,
 0.8436144174321903,
 -0.09046196714492456,
 -0.7351659999379173,
 0.7488328867021299,
 1.016860465608679,
 -0.7075446524211831,
 -0.9785186346036698,
 -0.9117987469751828,
 1.1461872090798941,
 0.06651134528445535,
 0.0359970970805411,
 -0.5760016754598903,
 -0.26367817409193073,
 -0.6388440378642762,
 0.4829112206428584,
 0.13471417670632202,
 0.6556380339924207,
 0.055020101514166454,
 -0.3692980052653513,
 -0.6145935723875064,
 0.09004824588207888,
 -0.29876770294531574,
 0.2635953984

In [38]:
dataset['tfidf'] = list(y_pred)

In [39]:
dataset.head()

,Page content,is_weekend,is_mon,article,title,article_clean,title_clean,num_char,num_words,num_sentence,unique_rate,nonstopword_rate,title_freq,# of img,# of link,# of total,dummy,tfidf
0,"<html><head><div class=""article-info""><span cl...",0,1,Note to humanity: One Direction fandom ai...,Soccer Star Gets Twitter Death Threats After T...,note humanity one direction fandom nothin mess...,soccer star get twitter death threat tackling ...,2294,382,37,0.704301,0.590476,0.075916,1,30,31,soccer star get twitter death threat tackling ...,-0.110850
1,"<html><head><div class=""article-info""><span cl...",0,0,Shortly after announcing a hardware upgrade f...,Google Glass Gets an Accessory Store,shortly announcing hardware upgrade google gla...,google glass get accessory store,489,84,6,0.696203,0.530201,0.178571,3,13,16,google glass get accessory store shortly annou...,0.286978
2,"<html><head><div class=""article-info""><span cl...",0,0,"Well, that was quick. Just hours after going ...",OUYA Gaming Console Already Sold Out on Amazon,well quick hour going sale u canada uk ouya ga...,ouya gaming console already sold amazon,620,105,13,0.808081,0.589286,0.142857,2,13,15,ouya gaming console already sold amazon well q...,-0.279798
3,"<html><head><div class=""article-info""><span cl...",0,0,Between Two Ferns: Oscar Buzz Edition Part 1...,'Between Two Ferns' Mocks Oscar Nominees,two fern oscar buzz edition part zach galifian...,two fern mock oscar nominee,670,113,7,0.787037,0.666667,0.097345,1,15,16,two fern mock oscar nominee two fern oscar buz...,0.141273
4,"<html><head><div class=""article-info""><span cl...",0,0,Ever since The Hurt Locker it seems like ...,'American Sniper' Trailer: Looks Like Eastwood...,ever since hurt locker seems like military the...,american sniper trailer look like eastwood may...,835,146,10,0.839416,0.608889,0.102740,1,10,11,american sniper trailer look like eastwood may...,-0.620511


In [40]:
dataset.to_csv('./data/test_3.csv')